# Azure AI Search 란?

Azure AI Search([이전의 "Azure Cognitive Search"](https://learn.microsoft.com/ko-kr/azure/search/whats-new#new-service-name))는 모든 규모의 고성능 애플리케이션을 위해 빌드된 포괄적인 고급 검색 기술을 갖춘 엔터프라이즈 지원 검색 및 검색 시스템입니다.

Azure AI Search는 Azure OpenAI Service와 Azure Machine Learning 간의 네이티브 LLM 통합을 통해 Azure에서 RAG 기반 애플리케이션을 빌드할 때 권장되는 기본 검색 시스템입니다.

Azure AI Search는 기존 시나리오와 GenAI 시나리오 모두에서 사용할 수 있습니다. 일반적인 사용 사례로는 기술 자료 인사이트(카탈로그 또는 문서 검색), 정보 검색(데이터 탐색), RAG(검색 보강 생성) 및 자동화가 포함됩니다.

검색 서비스를 만들 때 다음 기능을 사용합니다.

- 검색 인덱스를 통한 [벡터 검색](https://learn.microsoft.com/ko-kr/azure/search/vector-search-overview) 및 전체 텍스트 및 [하이브리드 검색](https://learn.microsoft.com/ko-kr/azure/search/hybrid-search-overview)에 사용되는 검색 엔진
- [통합 데이터 청크 및 벡터화](https://learn.microsoft.com/ko-kr/azure/search/vector-search-integrated-vectorization), 텍스트 [어휘 분석](https://learn.microsoft.com/ko-kr/azure/search/search-analyzers), 콘텐츠 추출 및 변환을 위한 [선택적 응용 AI](https://learn.microsoft.com/ko-kr/azure/search/cognitive-search-concept-intro)를 통한 풍부한 인덱싱
- [벡터 쿼리](https://learn.microsoft.com/ko-kr/azure/search/vector-search-how-to-query), 텍스트 검색, [하이브리드 쿼리](https://learn.microsoft.com/ko-kr/azure/search/hybrid-search-how-to-query), 유사 항목 검색, 자동 완성, 지역 검색 등을 위한 풍부한 쿼리 구문
- [의미론적 순위 지정](https://learn.microsoft.com/ko-kr/azure/search/semantic-search-overview), [채점 프로필](https://learn.microsoft.com/ko-kr/azure/search/index-add-scoring-profiles), [벡터 쿼리에 대한 양자화](https://learn.microsoft.com/ko-kr/azure/search/vector-search-how-to-configure-compression-storage) 및 런타임 시 쿼리 동작을 제어하기 위한 매개 변수를 사용하여 관련성 및 쿼리 성능 조정
- Azure 규모, 보안 및 도달률
- 데이터 레이어, 기계 학습 레이어, Azure AI 서비스 및 Azure OpenAI에서 Azure 통합

구조적으로 검색 서비스는 인덱싱되지 않은 데이터를 포함하는 외부 데이터 저장소와 검색 인덱스에 쿼리 요청을 보내고 응답을 처리하는 클라이언트 앱 사이에 배치됩니다.

![Azure AI 검색 아키텍처](https://learn.microsoft.com/ko-kr/azure/search/media/search-what-is-azure-search/azure-search.svg)

클라이언트 앱에서 검색 환경은 Azure AI 검색의 API를 통해 정의되며 관련성 튜닝, 의미 체계 순위 지정, 자동 완성, 동의어 일치, 유사 일치, 패턴 일치, 필터링 및 정렬이 이 환경에 포함될 수 있습니다.

Azure 플랫폼에서 Azure AI 검색은 Azure 데이터 원본에서 데이터 수집/검색을 자동화하는 인덱서 형태, 이미지 및 자연어 처리와 같은 Azure AI 서비스의 소모성 AI 또는 Azure Machine Learning에서 만들거나 Azure Functions 내부에서 래핑하는 사용자 지정 AI를 통합하는 기술 세트 형태의 다른 Azure 서비스와 통합할 수 있습니다.

> 📝 참고
>
> 더 자세한 내용은 [Azure AI 검색이란?](https://learn.microsoft.com/ko-kr/azure/search/search-what-is-azure-search) 을 참고해 주세요.

## 이 노트북의 목적

- 이 노트북에서는 어휘 검색을 위해 인덱스(index) 생성, 문서(document)를 입력하는 indexing과 기본적인 검색 방법을 학습합니다.

### library 및 변수 셋팅

In [2]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchIndex
)
import os

load_dotenv() # .env 에서 환경변수를 가져옵니다.

endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY")) if os.getenv("AZURE_SEARCH_ADMIN_KEY") else DefaultAzureCredential()

index_name = "hotels-quickstart"

### Index 생성

[SearchIndexClient](https://learn.microsoft.com/ko-kr/python/api/azure-search-documents/azure.search.documents.indexes.searchindexclient?view=azure-python)를 이용하여 index를 만듭니다.


In [ ]:
# 스키마 
fields = [
    SimpleField(name="HotelId", type=SearchFieldDataType.String, key=True),
    SearchableField(name="HotelName", type=SearchFieldDataType.String, sortable=True),
    SearchableField(name="Description", type=SearchFieldDataType.String, analyzer_name="en.lucene"),
    SearchableField(name="Description_fr", type=SearchFieldDataType.String, analyzer_name="fr.lucene"),
    SearchableField(name="Description_kr", type=SearchFieldDataType.String, analyzer_name="ko.lucene"),
    SearchableField(name="Category", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),

    SearchableField(name="Tags", collection=True, type=SearchFieldDataType.String, facetable=True, filterable=True),

    SimpleField(name="ParkingIncluded", type=SearchFieldDataType.Boolean, facetable=True, filterable=True, sortable=True),
    SimpleField(name="LastRenovationDate", type=SearchFieldDataType.DateTimeOffset, facetable=True, filterable=True, sortable=True),
    SimpleField(name="Rating", type=SearchFieldDataType.Double, facetable=True, filterable=True, sortable=True),

    ComplexField(name="Address", fields=[
        SearchableField(name="StreetAddress", type=SearchFieldDataType.String),
        SearchableField(name="City", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        SearchableField(name="StateProvince", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        SearchableField(name="PostalCode", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        SearchableField(name="Country", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
    ])
]

scoring_profiles = []
suggester = [{'name': 'sg', 'source_fields': ['Tags', 'Address/City', 'Address/Country']}]

# Index 생성
index = SearchIndex(name=index_name, fields=fields, suggesters=suggester, scoring_profiles=scoring_profiles)
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
result = index_client.create_or_update_index(index)
print(f' {result.name} 생성')


 hotels-quickstart 생성


### 입력할 document 데이터

In [17]:
documents = [
    {
    "@search.action": "upload",
    "HotelId": "1",
    "HotelName": "Secret Point Motel",
    "Description": "The hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.",
    "Description_fr": "L'hôtel est idéalement situé sur la principale artère commerciale de la ville en plein cœur de New York. A quelques minutes se trouve la place du temps et le centre historique de la ville, ainsi que d'autres lieux d'intérêt qui font de New York l'une des villes les plus attractives et cosmopolites de l'Amérique.",
    "Description_kr": "이 호텔은 뉴욕의 중심부에 있는 도시의 주요 상업 동맥에 이상적으로 위치해 있습니다. 몇 분 거리에 타임스 스퀘어와 뉴욕의 역사적인 중심지뿐만 아니라 뉴욕을 미국에서 가장 매력적이고 국제적인 도시로 만드는 다른 명소가 있습니다.",
    "Category": "Boutique",
    "Tags": [ "pool", "air conditioning", "concierge" ],
    "ParkingIncluded": "false",
    "LastRenovationDate": "1970-01-18T00:00:00Z",
    "Rating": 3.60,
    "Address": {
        "StreetAddress": "677 5th Ave",
        "City": "New York",
        "StateProvince": "NY",
        "PostalCode": "10022",
        "Country": "USA"
        }
    },
    {
    "@search.action": "upload",
    "HotelId": "2",
    "HotelName": "Twin Dome Motel",
    "Description": "The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts.",
    "Description_fr": "L'hôtel est situé dans une place du XIXe siècle, qui a été agrandie et rénovée aux plus hautes normes architecturales pour créer un hôtel moderne, fonctionnel et de première classe dans lequel l'art et les éléments historiques uniques coexistent avec le confort le plus moderne.",
    "Description_kr": "이 호텔은 19세기 광장에 위치해 있으며, 예술과 독특한 역사적 요소가 가장 현대적인 편안함과 공존하는 현대적이고 기능적인 일류 호텔을 만들기 위해 최고 수준의 건축 표준으로 확장 및 개조 되었습니다.",
    "Category": "Boutique",
    "Tags": [ "pool", "free wifi", "concierge" ],
    "ParkingIncluded": "false",
    "LastRenovationDate": "1979-02-18T00:00:00Z",
    "Rating": 3.60,
    "Address": {
        "StreetAddress": "140 University Town Center Dr",
        "City": "Sarasota",
        "StateProvince": "FL",
        "PostalCode": "34243",
        "Country": "USA"
        }
    },
    {
    "@search.action": "upload",
    "HotelId": "3",
    "HotelName": "Triple Landscape Hotel",
    "Description": "The Hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services.",
    "Description_fr": "L'hôtel est situé dans une place du XIXe siècle, qui a été agrandie et rénovée aux plus hautes normes architecturales pour créer un hôtel moderne, fonctionnel et de première classe dans lequel l'art et les éléments historiques uniques coexistent avec le confort le plus moderne.",
    "Description_kr": "호텔의 모든 레스토랑 서비스를 자문하고 감독하는 윌리엄 도우(William Dough)의 관리 하에 뛰어난 미식 경험을 제공하는 것이 특징입니다.",
    "Category": "Resort and Spa",
    "Tags": [ "air conditioning", "bar", "continental breakfast" ],
    "ParkingIncluded": "true",
    "LastRenovationDate": "2015-09-20T00:00:00Z",
    "Rating": 4.80,
    "Address": {
        "StreetAddress": "3393 Peachtree Rd",
        "City": "Atlanta",
        "StateProvince": "GA",
        "PostalCode": "30326",
        "Country": "USA"
        }
    },
    {
    "@search.action": "upload",
    "HotelId": "4",
    "HotelName": "Sublime Cliff Hotel",
    "Description": "Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace.",
    "Description_fr": "Le sublime Cliff Hotel est situé au coeur du centre historique de sublime dans un quartier extrêmement animé et vivant, à courte distance de marche des sites et monuments de la ville et est entouré par l'extraordinaire beauté des églises, des bâtiments, des commerces et Monuments. Sublime Cliff fait partie d'un Palace 1800 restauré avec amour.",
    "Description_kr": "수블림 클리프 호텔은 도시의 유적지와 랜드마크에서 도보로 가까운 거리에 있는 매우 활기차고 활기찬 수블림의 역사적인 중심지에 위치하고 있으며 교회, 건물, 상점 및 기념물의 특별한 아름다움으로 둘러싸여 있습니다. 수블림 클리프는 사랑스럽게 복원된 1800년 궁전의 일부입니다.",
    "Category": "Boutique",
    "Tags": [ "concierge", "view", "24-hour front desk service" ],
    "ParkingIncluded": "true",
    "LastRenovationDate": "1960-02-06T00:00:00Z",
    "Rating": 4.60,
    "Address": {
        "StreetAddress": "7400 San Pedro Ave",
        "City": "San Antonio",
        "StateProvince": "TX",
        "PostalCode": "78216",
        "Country": "USA"
        }
    }
]

### document indexing

문서를 인덱스에 인덱싱 합니다.

이를 위해 사용하는 SearchClien class에 대해 더 자세한 내용은 [SearchClient](https://learn.microsoft.com/ko-kr/python/api/azure-search-documents/azure.search.documents.searchclient?view=azure-python)에서 확인 할 수 있습니다.

In [5]:
search_client = SearchClient(
    endpoint=endpoint,
    credential=credential,
    index_name=index_name)

In [18]:
# 문서(document)를 indexing 합니다.
try:
    result = search_client.upload_documents(documents=documents)
    print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
    print (ex.message)

Upload of new document succeeded: True


### 검색

[SearchIndex](https://learn.microsoft.com/ko-kr/python/api/azure-search-documents/azure.search.documents.indexes.models.searchindex?view=azure-python)를 이용하여 검색을 수행 합니다.

search 함수에 대한 더 자세한 내용은 [search](https://learn.microsoft.com/ko-kr/python/api/azure-search-documents/azure.search.documents.searchclient?view=azure-python#azure-search-documents-searchclient-search)에서 확인 할 수 있습니다.

In [21]:
# 모든 document 를 검색하고, HotelName, Description 를 반환 합니다.
results =  search_client.search(query_type='simple',
    search_text="*" ,
    select='HotelName,Description',
    include_total_count=True)

# 검색 결과로 HotelName, Description 필드와 함께 검색 score도 표시합니다. 
print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

Total Documents Matching Query: 4
1.0
Triple Landscape Hotel
Description: The Hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services.
1.0
Twin Dome Motel
Description: The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts.
1.0
Sublime Cliff Hotel
Description: Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace.
1.0
Secret Point Motel
Description: The hotel is ideally located on the main commercial artery

### 검색어로 쿼리

검색어로 검색합니다.

In [15]:
# 'wifi'로 검색하고, HotelName, Description 를 반환 합니다.
results =  search_client.search(query_type='simple',
    search_text="century" ,
    select='HotelName,Description,Tags',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

Total Documents Matching Query: 1
0.71279967
Twin Dome Motel
Description: The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts.


In [6]:
# '클리프'로 검색하고, HotelName, Description 를 반환 합니다.
results =  search_client.search(query_type='simple',
    search_text="클리프" ,
    select='HotelName,Description,Tags',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

Total Documents Matching Query: 1
1.7906771
Sublime Cliff Hotel
Description: Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace.


### 필터(Filter)

검색 시 원하는 조건의 document를 필터링 하여 가져올 때 사용합니다.

In [22]:
# 'hotels' 로 검색하고, Rating 필드에서 4이상인 document를 가져온 후 Rating 필드 점수의 역순으로 정렬합니다.
results = search_client.search(
    search_text="hotels", 
    select='HotelId,HotelName,Rating', 
    filter='Rating gt 4', 
    order_by='Rating desc')

for result in results:
    print("{}: {} - {} rating".format(result["HotelId"], result["HotelName"], result["Rating"]))

3: Triple Landscape Hotel - 4.8 rating
4: Sublime Cliff Hotel - 4.6 rating


### 특정 필드에서 검색

특정 필드를 대상으로 검색을 수행 할 때 사용합니다.

In [23]:
# HotelName 필드에서만 검색합니다.

results = search_client.search(
    search_text="sublime", 
    search_fields=['HotelName'], 
    select='HotelId,HotelName')

for result in results:
    print("{}: {}".format(result["HotelId"], result["HotelName"]))

4: Sublime Cliff Hotel


### Facets

검색 시 입력한 조건에 대해 각가에 대한 결과 개수를 확인 할 수 있습니다. 

In [27]:
# 모든 document 를 검색하고, Category 별로 검색 결과 개수를 가져옵니다. 
results = search_client.search(search_text="*", facets=["Category"])

facets = results.get_facets()

for facet in facets["Category"]:
    print("{} - {}".format(facet['value'], facet['count']))

Boutique - 3
Resort and Spa - 1


### document id로 검색

indexing 된 document 들은 id를 가지게 되며, 해당 id로 문서를 검색 할 수 있습니다.
여기에 사용 될 필드는 인덱스 생성 시 key attribute를 true로 설정합니다.

이 예제에서는 HotelId를 key로 설정하였습니다.

``` python
SimpleField(name="HotelId", type=SearchFieldDataType.String, key=True),
```

[get_document](https://learn.microsoft.com/ko-kr/python/api/azure-search-documents/azure.search.documents.searchclient?view=azure-python#azure-search-documents-searchclient-autocomplete) 에서 더 자세한 내용을 확인 할 수 있습니다.

In [30]:
# 특정 document id로 검색
result = search_client.get_document(key="3")

print("Details for hotel '3' are:")
print("Name: {}".format(result["HotelName"]))
print("Rating: {}".format(result["Rating"]))
print("Category: {}".format(result["Category"]))

Details for hotel '3' are:
Name: Triple Landscape Hotel
Rating: 4.8
Category: Resort and Spa


### 쿼리 자동완성

쿼리 자동완성은 쿼리 생산성을 개선하기 위한 기술입니다. </br>
Azure AI Search에서는 부분 입력에 따라 용어 또는 구를 완성하는 자동 완성을 통해 이를 지원됩니다.</p>

index 생성 시 입력한 suggester의 source_fields 의 내용을 기반으로 자동완성 내용을 반환합니다.</br>

```
suggester = [{'name': 'sg', 'source_fields': ['Tags', 'Address/City', 'Address/Country']}]
```

[autocomplete](https://learn.microsoft.com/ko-kr/python/api/azure-search-documents/azure.search.documents.searchclient?view=azure-python#azure-search-documents-searchclient-autocomplete) 에서 더 자세한 내용을 확인 할 수 있습니다.

In [32]:
# Autocomplete a query
search_suggestion = 'sa'
results = search_client.autocomplete(
    search_text=search_suggestion, 
    suggester_name="sg",
    mode='twoTerms')

print("Autocomplete for:", search_suggestion)
for result in results:
    print (result['text'])

Autocomplete for: sa
san antonio
sarasota


### index 삭제

In [12]:
# index 삭제

try:
    result = index_client.delete_index(index_name)
    print ('Index', index_name, 'Deleted')
except Exception as ex:
    print (ex)

Index hotels-quickstart Deleted


In [15]:
# 인덱스가 지워졌는지 다시 확인.
# 정상적으로 삭제가 되었다면, index를 찾지 못해 오류가 발생합니다.
# 삭제가 되지 않앟다면 해당 인덱스가 나오게 됩니다.

try:
    result = index_client.get_index(index_name)
    print (result)
except Exception as ex:
    print (ex)

() No index with the name 'hotels-quickstart' was found in the service 'dev-ai-search-west-us-01'.
Code: 
Message: No index with the name 'hotels-quickstart' was found in the service 'dev-ai-search-west-us-01'.
